# Import libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pylab
import pytrip as pt
import copy
import matplotlib as mpl

In [ ]:
from matplotlib import colors, cm
from numpy import ma
from pytrip.volhist import VolHist 
from itertools import cycle
cycol = cycle('bgrcmk')

In [ ]:
style_param = {'legend.fontsize': '20',
         'xtick.direction' : 'in',  
         'ytick.direction' : 'in', 
         'xtick.top' : True, 
         'figure.figsize': (10,6),
         'axes.labelsize': '26',
         'axes.titlesize':'26',
         'xtick.labelsize':'20',
         'ytick.labelsize':'20',
         'xtick.major.pad':'16',
         'ytick.major.pad':'16'}
    
pylab.rcParams.update(style_param)

# Functions

In [ ]:
def rbe_wedenberg(dose, let, abx,q):
    """
    Wedenberg proton RBE model
    input parameters may be either numpy.array or scalars
    TODO: handle Cube() class directly
    :params dose: physical proton dose in [Gy]
    :params let: LETd in [keV/um] (protons only)
    :params abx: alpha_x / beta_x [Gy]
    :returns: RBE for the given parameters
    :ref: http://dx.doi.org/10.3109/0284186X.2012.705892
    """

    _apx = 1.000 + q * let / abx
    _sbpx = 1.0

    rbe = _rbe_apx(dose, _apx, _sbpx, abx)
    return rbe

In [ ]:
def _rbe_apx(dose, apx, sbpx, abx, dzero=0.0):
    """
    :params dose: proton dose      [Gy]
    :params apx: alpha_p / alpha_x [dimensionless] RBE_max = ap/ax when (dose -> 0 Gy)
    :params sbpx: beta_p / beta_x  [dimensionless] RBE_min = sqrt(bp/bx) when (dose -> inf Gy)
    :params abx: alpha_x / beta_x  [Gy]
    :params dzero: what to return in case of dose is zero (which would cause division by zero)
    """

    _rbe = 1.0 / (2.0 * dose)
    if hasattr(_rbe, '__iter__'):
        _rbe[_rbe == np.inf] = dzero
    else:
        if _rbe == np.inf:
            return dzero
    delta = abx * abx + 4. * abx * apx * dose + 4. * sbpx * sbpx * dose * dose
    delta *= (delta > 0)
    _rbe *= (np.sqrt(delta) - abx)
    return _rbe

In [ ]:
def biological_dose (q,ab,dos_cube,let_cube):
    result = pt.DosCube()
    result.cube = rbe_wedenberg(dos_cube.cube, let_cube.cube, ab, q) * dos_cube.cube
    return result

# Prescribed dose and number of fractionation

In [ ]:
## Choose alpha/beta ratio == 2.0 or 10.0
dose_pres = 55.8 
fraction=31.
dose_per_fr=dose_pres/fraction
ab=2.
bins=256

In [ ]:
def calculate_dvh (qval,mask,dose_cube,let_cube, ab=ab,bins=bins,dose_pres=dose_pres,fraction=dose_per_fr):
    
    biol_dose= rbe_wedenberg(dose=dose_cube.cube[mask], let=let_cube.cube[mask], abx=ab, q=qval) * dose_cube.cube[mask]

    _xrange = (0.0,2.4)
    _hist, x = np.histogram((biol_dose), bins=1024, range=_xrange,density=True)
    _fhist = _hist[::-1]  # reverse histogram, so first element is for highest dose
    _fhist = np.cumsum(_fhist)
    _hist = _fhist[::-1]  # flip back again to normal representation

    y = 100.0 * _hist / _hist[0]  # volume histograms always plot the right edge of bin, since V(D < x_pos).
    y = np.insert(y, 0, 100.0, axis=0)  # but the leading bin edge is always at V = 100.0%
    return(x,y)    

In [ ]:
def plot_dvh ( dose_cube, bins ):
   

    _hist, x = np.histogram(dose_cube, bins=bins, density=True)
    _fhist = _hist[::-1]  # reverse histogram, so first element is for highest dose
    _fhist = np.cumsum(_fhist)
    _hist = _fhist[::-1]  # flip back again to normal representation

    y = 100.0 * _hist / _hist[0]  # volume histograms always plot the right edge of bin, since V(D < x_pos).
    y = np.insert(y, 0, 100.0, axis=0)  # but the leading bin edge is always at V = 100.0%
    return(x,y)    

In [ ]:
def plot_dvh_xrange ( dose_cube,bins = 1024):
   
    _xrange=(0,2.4)
    _hist, x = np.histogram(dose_cube, bins=bins,range=_xrange, density=True)
    _fhist = _hist[::-1]  # reverse histogram, so first element is for highest dose
    _fhist = np.cumsum(_fhist)
    _hist = _fhist[::-1]  # flip back again to normal representation

    y = 100.0 * _hist / _hist[0]  # volume histograms always plot the right edge of bin, since V(D < x_pos).
    y = np.insert(y, 0, 100.0, axis=0)  # but the leading bin edge is always at V = 100.0%
    return(x,y)  

# Read data

In [ ]:
#original plan from CCB
patient_org = 'data/plan1'
#modyfied plan (5 degrees)
patient_mod = 'data/plan2'

In [ ]:
#q
open_fname = os.path.join('tmp','distrib_q.h5')
q = pd.read_hdf(open_fname, 'data_1')

# Patient: PyTrip Data

In [ ]:
## FIRST CASE

patient_name = '/inspire07pancreas'
file_corename = os.path.join(patient_org+patient_name)

dos_cube = pt.DosCube()
dos_cube.read(file_corename)
dos_cube.cube = dos_cube.cube.astype(np.float64)
dos_cube.cube /= 1000.0
dos_cube.cube *= dose_per_fr/1.1

let_cube = pt.LETCube()
let_cube.read(file_corename)

ctx_cube = pt.CtxCube()
ctx_cube.read(file_corename+'.ctx')

vdx_cube = pt.VdxCube(ctx_cube)
vdx_cube.read(file_corename + '.vdx')

# index of point in original plan with highest dose
imax = np.unravel_index(dos_cube.cube.argmax(), dos_cube.cube.shape)

In [ ]:
imax

In [ ]:
#SECOND CASE

patient_name = '/inspire07pancreas'
file_corename = os.path.join(patient_mod+patient_name)

dos_cube2 = pt.DosCube()
dos_cube2.read(file_corename)
dos_cube2.cube = dos_cube2.cube.astype(np.float64)
dos_cube2.cube /= 1000.0
dos_cube2.cube *= dose_per_fr/1.1

let_cube2 = pt.LETCube()
let_cube2.read(file_corename)

ctx_cube2 = pt.CtxCube()
ctx_cube2.read(file_corename)

vdx_cube2 = pt.VdxCube(ctx_cube2)
vdx_cube2.read(file_corename + '.vdx')

In [ ]:
ctx_cube.slice_pos[imax[0]]

In [ ]:
ctx_cube.yoffset

In [ ]:
ctx_cube.dimy

In [ ]:
dos_cube.pixel_size

Organs

In [ ]:
print(vdx_cube.voi_names())

In [ ]:
contour_data = vdx_cube.get_voi_by_name('iCTV_5580').get_slice_at_pos(ctx_cube.slice_pos[imax[0]]).contours[0].contour
contour_liver = vdx_cube.get_voi_by_name('liver').get_slice_at_pos(ctx_cube.slice_pos[imax[0]]).contours[0].contour
contour_kidney_L = vdx_cube.get_voi_by_name('kidney_L').get_slice_at_pos(ctx_cube.slice_pos[imax[0]]).contours[0].contour
contour_kidney_R = vdx_cube.get_voi_by_name('kidney_R').get_slice_at_pos(ctx_cube.slice_pos[imax[0]]).contours[0].contour
contour_spinalCord = vdx_cube.get_voi_by_name('SpinalCord').get_slice_at_pos(ctx_cube.slice_pos[imax[0]]).contours[0].contour
contour_spleen = vdx_cube.get_voi_by_name('spleen').get_slice_at_pos(ctx_cube.slice_pos[110]).contours[0].contour
contour_duodenum = vdx_cube.get_voi_by_name('duodenum').get_slice_at_pos(ctx_cube.slice_pos[90]).contours[0].contour

In [ ]:
x = [item[0] / 10. for item in contour_data]
y = [item[1] / 10. for item in contour_data]
x_liv = [item[0] / 10. for item in contour_liver]
y_liv = [item[1] / 10. for item in contour_liver]
x_kidney_L = [item[0] / 10. for item in contour_kidney_L]
y_kidney_L = [item[1] / 10. for item in contour_kidney_L]
x_kidney_R = [item[0] / 10. for item in contour_kidney_R]
y_kidney_R = [item[1] / 10. for item in contour_kidney_R]
x_spleen = [item[0] / 10. for item in contour_spleen]
y_spleen = [item[1] / 10. for item in contour_spleen]
x_spinalCord = [item[0] / 10. for item in contour_spinalCord]
y_spinalCord = [item[1] / 10. for item in contour_spinalCord]
x_duodenum = [item[0] / 10. for item in contour_duodenum]
y_duodenum = [item[1] / 10. for item in contour_duodenum]

Visualization dose in patient dor two plans

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(22,12))

pixel_mm = dos_cube.pixel_size/10.
pixel_mm2 = dos_cube2.pixel_size/10.


im11 = ax1.imshow(ctx_cube.cube[imax[0],:,:],
                cmap=plt.cm.gray,
                interpolation='nearest',  
                origin="lower",
                 extent=[ctx_cube.xoffset/10,ctx_cube.xoffset/10 + ctx_cube.dimx*pixel_mm,
                         ctx_cube.yoffset/10,ctx_cube.yoffset/10 + ctx_cube.dimy*pixel_mm])

im22 = ax2.imshow(ctx_cube2.cube[imax[0],:,:],
                cmap=plt.cm.gray,
                interpolation='nearest',  
                origin="lower",
                extent=[ctx_cube.xoffset/10,ctx_cube.xoffset/10 + ctx_cube.dimx*pixel_mm2,
                         ctx_cube.yoffset/10,ctx_cube.yoffset/10 + ctx_cube.dimy*pixel_mm2])

cmap1 = copy.copy(mpl.cm.get_cmap("jet"))
cmap1 = plt.cm.jet
cmap1.set_under("k", alpha=0.0) 
cmap1.set_over("k", alpha=0.0)
cmap1.set_bad("k", alpha=0.0) 

dmin = dos_cube.cube.min()
dmax = dos_cube.cube.max() * 1.1
tmpdat = ma.masked_where(dos_cube.cube[imax[0],:,:] <= dmin, dos_cube.cube[imax[0],:,:])  # Sacrificial goat

# plot new data cube
im1 = ax1.imshow(
    tmpdat,
    cmap=cmap1,
    norm=colors.Normalize(vmin=0, vmax=dmax, clip=False),
    alpha=0.5,
    interpolation='nearest',  # each pixel will get colour of nearest neighbour, useful when displaying
    origin="lower",  # ['upper' | 'lower']:
    extent=[ctx_cube.xoffset/10,ctx_cube.xoffset/10 + ctx_cube.dimx*pixel_mm,
ctx_cube.yoffset/10,ctx_cube.yoffset/10 + ctx_cube.dimy*pixel_mm])


dmin2 = dos_cube2.cube.min()
dmax2 = dos_cube2.cube.max() * 1.1
tmpdat2 = ma.masked_where(dos_cube2.cube[imax[0],:,:] <= dmin2, dos_cube2.cube[imax[0],:,:])  # Sacrificial goat

im2 = ax2.imshow(tmpdat2, 
                cmap=cmap1,
                norm=colors.Normalize(vmin=0, vmax=dmax2, clip=False),
                alpha=0.5,
                interpolation='nearest',
                 origin="lower",
                 extent=[ctx_cube2.xoffset/10,ctx_cube2.xoffset/10 + ctx_cube2.dimx*pixel_mm2,
                         ctx_cube2.yoffset/10,ctx_cube2.yoffset/10 + ctx_cube2.dimy*pixel_mm2])


ax1.plot(x,y,c='y', label = "CTV")
ax2.plot(x,y,c='y', label = "CTV")

ax1.plot(x_kidney_L,y_kidney_L, c='orange',label = "left kidney")
ax2.plot(x_kidney_L,y_kidney_L, c='orange',label = "left kidney")
ax1.plot(x_kidney_R,y_kidney_R, c='r',label = "right kidney")
ax2.plot(x_kidney_R,y_kidney_R, c='r',label = "right kidney")

ax1.plot(x_liv,y_liv,c='g',label = "liver")
ax2.plot(x_liv,y_liv,c='g',label='liver')

ax1.plot(x_spleen,y_spleen, c='blue',label = "spleen")
ax2.plot(x_spleen,y_spleen, c='blue',label = "spleen")
ax1.plot(x_spinalCord,y_spinalCord, c='yellow',label = "spinalCord")
ax2.plot(x_spinalCord,y_spinalCord, c='yellow',label = "spinalCord")

ax1.plot(x_duodenum,y_duodenum, c='purple',label = "duodenum")
ax2.plot(x_duodenum,y_duodenum, c='purple',label = "duodenum")

ax1.set_xlabel('X [cm]',fontsize=22)
ax1.set_ylabel('Y [cm]',fontsize=22)
ax2.set_xlabel('X [cm]',fontsize=22)
ax2.set_ylabel('Y [cm]',fontsize=22)

##Plan 1 
ax1.set_title("Plan 1\n\n\n\n",fontsize=26)
ax2.set_title("Plan 2\n\n\n\n",fontsize=26)

ax1.legend(title = "Irradiated areas",loc='best',bbox_to_anchor=(2 , 1.28),fancybox=True, framealpha=0.6, title_fontsize=22,ncol=4)
#ax2.legend(title = "Irradiated areas",loc='best',bbox_to_anchor=(1, 1.2),fancybox=True, framealpha=0.6, title_fontsize=22,ncol=2)

ax1.invert_yaxis()
ax2.invert_yaxis()

ax1.plot([-25, 0], [-10,-25], 'w-')
ax1.text(-25,-15, r'$240^0$',fontsize=22, color='w')
ax1.plot([0,25], [-25,-5], 'w-')
ax1.text(18,-11, r'$130^0$',fontsize=22, color='w')


ax2.plot([-25, 0], [-13,-25], 'w-')
ax2.text(-25,-17, r'$245^0$',fontsize=22, color='w')

ax2.plot([0,25], [-25,-8], 'w-')
ax2.text(18,-13, r'$125^0$',fontsize=22, color='w')

fig.colorbar(im1, ax=ax1, label ="Dose [Gy]", shrink = 0.5).set_label(label="Dose [Gy]", size=24, labelpad=-15, y=1.15, rotation=0)
fig.colorbar(im2, ax=ax2, label ="Dose [Gy]", shrink = 0.5 ).set_label(label="Dose [Gy]", size=24, labelpad=-15, y=1.15, rotation=0)

In [ ]:
#Dose difference in patient for two plans
fig, (ax1) = plt.subplots(1,1, figsize=(16,12))
pixel_mm = dos_cube.pixel_size/10.

im11 = ax1.imshow(ctx_cube2.cube[imax[0],:,:],
                cmap=plt.cm.gray,
                interpolation='nearest',  
                origin="lower",
                 extent=[ctx_cube.xoffset/10,ctx_cube.xoffset/10 + ctx_cube.dimx*pixel_mm,
                         ctx_cube.yoffset/10,ctx_cube.yoffset/10 + ctx_cube.dimy*pixel_mm])

tmpdat = ma.masked_where((dos_cube2.cube[imax[0],:,:] - dos_cube.cube[imax[0],:,:]) <= -1.25,
                         (dos_cube2.cube[imax[0],:,:] - dos_cube.cube[imax[0],:,:]) ) # Sacrificial goat

im1 = ax1.imshow(dos_cube2.cube[imax[0],:,:] - dos_cube.cube[imax[0],:,:], 
                  cmap=cm.seismic,vmin=-1.25,vmax=1.25,interpolation='nearest', 
                  origin="lower",              #upper
                  alpha=0.4,
                  extent=[ctx_cube.xoffset/10,ctx_cube.xoffset/10 + ctx_cube.dimx*pixel_mm,
                          ctx_cube.yoffset/10,ctx_cube.yoffset/10 + ctx_cube.dimy*pixel_mm])


ax1.plot(x,y,c='r', label = "CTV")
ax1.plot(x_kidney_L,y_kidney_L, c='orange',label = "left kidney")
ax1.plot(x_kidney_R,y_kidney_R, c='red',label = "right kidney")
ax1.plot(x_liv,y_liv,c='green',label = "liver")
ax1.plot(x_spleen,y_spleen, c='blue',label = "spleen")
ax1.plot(x_spinalCord,y_spinalCord, c='yellow',label = "spinalCord")
ax1.plot(x_duodenum,y_duodenum, c='purple',label = "duodenum")

ax1.set_xlabel('X [cm]',fontsize=15)
ax1.set_ylabel('Y [cm]',fontsize=15)
ax1.set_title("Differences\n\n\n\n",fontsize=20)
ax1.legend(title = "Irradiated areas",loc='best',bbox_to_anchor=(1.05, 1.15),fancybox=True, title_fontsize=22,framealpha=0.6,ncol=4)

ax1.invert_yaxis()

cbar= fig.colorbar(im1, ax=ax1,ticks=[-1, 0, 1], shrink = 0.5,extend='both')
cbar.set_label(label="Dose [Gy]", size=18, labelpad=-8, y=1.05, rotation=0)
cbar.set_ticks([-1.25,-1.,0,1.,1.25])
cbar.set_ticklabels(['-1.25','Plan 1','0', 'Plan 2','1.25'])
cbar.ax.tick_params(labelsize=22)

Uncetainties RBE for Wedenberg and Wedenberg B transversal sections

In [ ]:
#mean q value with 85% CI
mean_W, std_W = 0.434, 1.44*0.0301
mean_WB, down_WB, up_WB = q.q.mean(), q.q.quantile(0.075), q.q.quantile(0.925)

In [ ]:
#Uncetainties RBE for Wedenberg transversal section

err_W=[]
err_up_W=[]
err_down_W=[]

for dose, let in zip(dos_cube.cube[imax[0],imax[1],:],let_cube.cube[imax[0],imax[1],:]):
    
    if dose>0:
        df=rbe_wedenberg(dose,let,ab,mean_W)*dose
        df_up=rbe_wedenberg(dose,let,ab,mean_W+std_W)*dose
        df_down=rbe_wedenberg(dose,let,ab,mean_W-std_W)*dose

        err_W.append(df *100)
        err_up_W.append(df_up *100)
        err_down_W.append(df_down *100)

    else:
        err_W.append(0)
        err_up_W.append(0)
        err_down_W.append(0)

In [ ]:
#Uncetainties RBE for Wedenberg transversal section
plt.figure(figsize=(22,10))

plt.plot(err_W,'r', linewidth=2., label = "mean")
plt.plot(err_up_W,'r.', linewidth=2.)
plt.plot(err_down_W,'r.', linewidth=2.)

#Jak zamienić na [cm] na x-ach
plt.ylabel("Biological Dose[Gy(RBE)]", fontsize = 35)
plt.xlabel("x-coordinate", fontsize = 35)
plt.legend(loc = "best",  fontsize = 35)

plt.yticks(fontsize = 15)
plt.xlim(210,330)
plt.grid()
plt.minorticks_on()
plt.grid(which='minor', linestyle=':', linewidth='0.2', color='black')

plt.rcParams['xtick.direction'] = 'in' 
plt.rcParams['ytick.direction'] = 'in' 
plt.rcParams['xtick.top'] = True 
plt.rcParams['ytick.right'] = True

In [ ]:
#Uncetainties RBE for WedenbergB transversal section

err_WB=[]
err_up_WB=[]
err_down_WB=[]

for dose, let in zip(dos_cube.cube[imax[0],imax[1],:],let_cube.cube[imax[0],imax[1],:]):
    
    if dose>0:
        df=rbe_wedenberg(dose,let,ab,q.mean())*dose
        df_up=rbe_wedenberg(dose,let,ab,q.mean()+q.std())*dose
        df_down=rbe_wedenberg(dose,let,ab,q.mean()-q.std())*dose

        err_WB.append(df *100)
        err_up_WB.append(df_up *100)
        err_down_WB.append(df_down *100)

    else:
        err_WB.append(0)
        err_up_WB.append(0)
        err_down_WB.append(0)

In [ ]:
#Uncetainties RBE for WedenbergB transversal section

plt.figure(figsize=(22,10))

plt.plot(err_WB,'g', linewidth=2., label = "mean")
plt.plot(err_up_WB,'g.', linewidth=2.)
plt.plot(err_down_WB,'g.', linewidth=2.)

plt.ylabel("Biological Dose[Gy(RBE)]", fontsize = 35)
plt.yticks(fontsize = 15)


#Jak zamienić na [cm] na x-ach

plt.xlabel("x-coordinate", fontsize = 35)
plt.legend(loc = "best",  fontsize = 35)

plt.xlim(210,330)
plt.grid()
plt.minorticks_on()
plt.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
#plt.axvspan(80*dos_cube.pixel_size,106*dos_cube.pixel_size, alpha=0.3, color='gray')


plt.rcParams['xtick.direction'] = 'in' 
plt.rcParams['ytick.direction'] = 'in' 
plt.rcParams['xtick.top'] = True 
plt.rcParams['ytick.right'] = True

Compare uncertainties for 2 plans

In [ ]:
plt.figure(figsize=(22,10))


plt.plot(err_W,'r', linewidth=2., label = "Wedenberg model")
plt.plot(err_up_W,'r.', linewidth=2.)
plt.plot(err_down_W,'r.', linewidth=2.)

plt.plot(err_WB,'g', linewidth=2., label = "WedenbergB model")
plt.plot(err_up_WB,'g.', linewidth=2.)
plt.plot(err_down_WB,'g.', linewidth=2.)

plt.ylabel("Biological Dose\nuncertainties [%]", fontsize = 35)
plt.yticks(fontsize = 15)


#Change to cm on axis

plt.xlabel("x-coordinate", fontsize = 35)
plt.legend(loc = "best",  fontsize = 35)

plt.xlim(210,330)
plt.grid()
plt.minorticks_on()
plt.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
#plt.axvspan(80*dos_cube.pixel_size,106*dos_cube.pixel_size, alpha=0.3, color='gray')


plt.rcParams['xtick.direction'] = 'in' 
plt.rcParams['ytick.direction'] = 'in' 
plt.rcParams['xtick.top'] = True 
plt.rcParams['ytick.right'] = True

# DVH - volume_histogram() function

In [ ]:
#Clinical model: factor 1.1 : Plan 1 i 2
#normalization for 1.8 Gy 
dvh_clinical = VolHist.volume_histogram(dos_cube.cube,voi=vdx_cube.get_voi_by_name('iCTV_5580'),bins=256)
dvh2_clinical = VolHist.volume_histogram(dos_cube2.cube,voi=vdx_cube.get_voi_by_name('iCTV_5580'),bins=256)

In [ ]:
# static volume_histogram(data_cube, voi=None, bins=256)
dvh = VolHist(dos_cube,voi=vdx_cube.get_voi_by_name('iCTV_5580')).volume_histogram(dos_cube.cube,voi=vdx_cube.get_voi_by_name('iCTV_5580'),bins=256)
dvh2 = VolHist(dos_cube2,voi=vdx_cube.get_voi_by_name('iCTV_5580')).volume_histogram(dos_cube2.cube,voi=vdx_cube.get_voi_by_name('iCTV_5580'),bins=256)   #inna metoda

In [ ]:
plt.figure(figsize=(22,10))

plt.plot(dvh[0], dvh[1], 'k:', linewidth=4, label = "Plan 1")
plt.plot(dvh2[0], dvh2[1], 'b', linewidth=4, label = "Plan 2")

plt.axvline(dose_per_fr/1.1, label="Dose per fraction")

#Graph visualisation
plt.title("Physical dose in CTV"+"\n\n\n")
plt.ylabel("Volume[%]")
plt.xlabel("\nDose[Gy]")

plt.xlim(0,None)

plt.legend(loc='best',title_fontsize=22,fancybox=True, shadow=True,ncol=4)

plt.grid()
plt.minorticks_on()
plt.grid(which='minor', linestyle=':', linewidth='0.2', color='black')


# Biological Dose

In [ ]:
organs = ['iCTV_5580','liver','spleen','kidney_L','kidney_R','pancreas','Spinalcord','duodenum']

In [ ]:
#długo sie liczy -> kilka minut, a za kolejną iteracją już szybko

dvh_wedB = dict()
dvh_wedB["Plan1"] = dict()
dvh_wedB["Plan2"] = dict()

for organ in organs:
    target_voi = vdx_cube.get_voi_by_name(organ)
    voi_cube = target_voi.get_voi_cube()
    mask = (voi_cube.cube == 1000)
    
    dvh_wedB["Plan1"][organ] = dict()
    dvh_wedB["Plan1"][organ]["mean"] = calculate_dvh(qval=mean_WB,mask=mask,dose_cube=dos_cube,let_cube=let_cube)
    dvh_wedB["Plan1"][organ]["down"] = calculate_dvh(qval=down_WB,mask=mask,dose_cube=dos_cube,let_cube=let_cube)
    dvh_wedB["Plan1"][organ]["up"] = calculate_dvh(qval=up_WB,mask=mask,dose_cube=dos_cube,let_cube=let_cube)
    
    dvh_wedB["Plan2"][organ] = dict()
    dvh_wedB["Plan2"][organ]["mean"] = calculate_dvh(qval=mean_WB,mask=mask,dose_cube=dos_cube2,let_cube=let_cube2)
    dvh_wedB["Plan2"][organ]["down"] = calculate_dvh(qval=down_WB,mask=mask,dose_cube=dos_cube2,let_cube=let_cube2)
    dvh_wedB["Plan2"][organ]["up"] = calculate_dvh(qval=up_WB,mask=mask,dose_cube=dos_cube2,let_cube=let_cube2)

In [ ]:
dvh_wed = dict()
dvh_wed["Plan1"] = dict()
dvh_wed["Plan2"] = dict()

for organ in organs:
    
    target_voi = vdx_cube.get_voi_by_name(organ)
    voi_cube = target_voi.get_voi_cube()
    mask = (voi_cube.cube == 1000)
    
    dvh_wed["Plan1"][organ] = dict()
    dvh_wed["Plan2"][organ] = dict()

    if organ == 'iCTV_5580':
        dvh_wed["Plan1"][organ]["mean"] = calculate_dvh(qval=mean_W,mask=mask,dose_cube=dos_cube,let_cube=let_cube)
        dvh_wed["Plan1"][organ]["down"] =calculate_dvh(qval=mean_W-std_W,mask=mask,dose_cube=dos_cube,let_cube=let_cube)
        dvh_wed["Plan1"][organ]["up"] = calculate_dvh(qval=mean_W+std_W,mask=mask,dose_cube=dos_cube,let_cube=let_cube)
        
        dvh_wed["Plan2"][organ]["mean"] = calculate_dvh(qval=mean_W,mask=mask,dose_cube=dos_cube2,let_cube=let_cube2)
        dvh_wed["Plan2"][organ]["down"] =calculate_dvh(qval=mean_W-std_W,mask=mask,dose_cube=dos_cube2,let_cube=let_cube2)
        dvh_wed["Plan2"][organ]["up"] = calculate_dvh(qval=mean_W+std_W,mask=mask,dose_cube=dos_cube2,let_cube=let_cube2)
    else:
        dvh_wed["Plan1"][organ]["mean"] = calculate_dvh(qval=mean_W,mask=mask,dose_cube=dos_cube,let_cube=let_cube)
        dvh_wed["Plan2"][organ]["mean"] = calculate_dvh(qval=mean_W,mask=mask,dose_cube=dos_cube2,let_cube=let_cube2)

In [ ]:
#Plan 1 for Wedenberg and WedenbergB models

fig,ax = plt.subplots(figsize=[22,10])
ax2=ax.twinx()

##Plan 1
for organ in organs:
#WedenbergB
    col=next(cycol)
    ax.plot(dvh_wedB["Plan1"][organ]['mean'][0],dvh_wedB["Plan1"][organ]['mean'][1],c=col, label = organ)
    ax.plot(dvh_wedB["Plan1"][organ]['up'][0],dvh_wedB["Plan1"][organ]['up'][1],".",c=col,markersize=1)
    ax.plot(dvh_wedB["Plan1"][organ]['down'][0],dvh_wedB["Plan1"][organ]['down'][1],".",c=col,markersize=1)
#Wedenberg 
    col=next(cycol)
    ax2.plot(dvh_wed["Plan1"][organ]['mean'][0],dvh_wed["Plan1"][organ]['mean'][1],c=col, label = organ)


##graph settings
ax.set_title("DVH\nPlanning using Wedenberg extended model B and Wedenberg model\n\n\n\n\n\n")
ax.set_ylabel("Volume [%]")
ax.set_xlabel("\nDose [Gy(RBE)]")

ax.legend(loc='upper center', bbox_to_anchor=(0.25, 1.3),title_fontsize=22,fancybox=True, shadow=True,ncol=2)
ax.get_legend().set_title("Wedenberg B")
ax2.legend(loc='upper center', bbox_to_anchor=(0.75, 1.3),title_fontsize=22,fancybox=True, shadow=True,ncol=2)
ax2.get_legend().set_title("Wedenberg")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='black')

ax.set_xlim(0,2.4)
ax2.set_xlim(0,2.4)

In [ ]:
#Compare two plans for WedenbergB model

fig,ax = plt.subplots(figsize=[22,10])
ax2=ax.twinx()
##Plan 1
for organ in organs:
    col=next(cycol)
    ax.plot(dvh_wedB["Plan1"][organ]['mean'][0],dvh_wedB["Plan1"][organ]['mean'][1],c=col, label = organ)
    ax.plot(dvh_wedB["Plan1"][organ]['up'][0],dvh_wedB["Plan1"][organ]['up'][1],".",c=col,markersize=1)
    ax.plot(dvh_wedB["Plan1"][organ]['down'][0],dvh_wedB["Plan1"][organ]['down'][1],".",c=col,markersize=1)
#Plan 2
    col=next(cycol)
    ax2.plot(dvh_wedB["Plan2"][organ]['mean'][0],dvh_wedB["Plan2"][organ]['mean'][1],c=col, label = organ)
    ax2.plot(dvh_wedB["Plan2"][organ]['up'][0],dvh_wedB["Plan2"][organ]['up'][1],".",c=col,markersize=1)
    ax2.plot(dvh_wedB["Plan2"][organ]['down'][0],dvh_wedB["Plan2"][organ]['down'][1],".",c=col,markersize=1)

##graph settings
ax.set_title("DVH\nPlanning using Wedenberg extended model B\n\n\n\n\n\n")
ax.set_ylabel("Volume [%]")
ax.set_xlabel("\nDose [Gy(RBE)]")

#plt.xlim(1.5,2.2)
ax.legend(loc='upper center', bbox_to_anchor=(0.25, 1.3),title_fontsize=22,fancybox=True, shadow=True,ncol=2)
ax.get_legend().set_title("Plan 1")

ax2.legend(loc='upper center', bbox_to_anchor=(0.75, 1.3),title_fontsize=22,fancybox=True, shadow=True,ncol=2)
ax2.get_legend().set_title("Plan2")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
ax.axvspan(0,0.7, alpha=0.3, color='gray')

ax.set_xlim(0,2.4)
ax2.set_xlim(0,2.4)

In [ ]:
#Compare two plans for WedenbergB model -> ZOOM

#choode organ to compare
organs2 = ['liver','spleen','kidney_L','kidney_R','Spinalcord','duodenum']
fig,ax = plt.subplots(figsize=[22,10])
ax2=ax.twinx()

##Plan 1
for organ in organs2:
    col=next(cycol)
    ax.plot(dvh_wedB["Plan1"][organ]['mean'][0],dvh_wedB["Plan1"][organ]['mean'][1],c=col, label = organ)
    ax.plot(dvh_wedB["Plan1"][organ]['up'][0],dvh_wedB["Plan1"][organ]['up'][1],".",c=col,markersize=1)
    ax.plot(dvh_wedB["Plan1"][organ]['down'][0],dvh_wedB["Plan1"][organ]['down'][1],".",c=col,markersize=1)
#Plan 2
    col=next(cycol)
    ax2.plot(dvh_wedB["Plan2"][organ]['mean'][0],dvh_wedB["Plan2"][organ]['mean'][1],c=col, label = organ)
    ax2.plot(dvh_wedB["Plan2"][organ]['up'][0],dvh_wedB["Plan2"][organ]['up'][1],".",c=col,markersize=1)
    ax2.plot(dvh_wedB["Plan2"][organ]['down'][0],dvh_wedB["Plan2"][organ]['down'][1],".",c=col,markersize=1)


##graph settings
ax.set_title("DVH\nPlanning using Wedenberg extended model B\n\n\n\n\n\n")
ax.set_ylabel("Volume [%]")
ax.set_xlabel("\nDose [Gy(RBE)]")

ax.legend(loc='upper center', bbox_to_anchor=(0.25, 1.3),title_fontsize=22,fancybox=True, shadow=True,ncol=2)
ax.get_legend().set_title("Plan 1")

ax2.legend(loc='upper center', bbox_to_anchor=(0.75, 1.3),title_fontsize=22,fancybox=True, shadow=True,ncol=2)
ax2.get_legend().set_title("Plan2")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='black')

ax.set_xlim(0,0.7)
ax2.set_xlim(0,0.7)
ax.set_ylim(0,50)